In [3]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
import glm_model as gm
from glm_utils import *
import tensorflow as tf

%load_ext autoreload
%autoreload 2
%matplotlib inline

In [4]:
#finally get rid of the ugly ass allen code. We import the pickled design 
#matrix stimulus, which is PCA projection of the images, and the downsampled
#traces. 

import pickle

pkl_file = open('./boc/formatted/LMcelldata')
data = pickle.load(pkl_file)
pkl_file.close()

pkl_file = open('./boc/formatted/LMstimdata')
stim = pickle.load(pkl_file)
pkl_file.close()

pkl_file = open('./boc/formatted/runspeed')
run_speed = pickle.load(pkl_file)
pkl_file.close()

In [5]:
#get the number of features, and the number of training examples. 
l, n =  stim.shape

In [ ]:
from statsmodels.tsa.tsatools import lagmat
from sklearn.cross_validation import train_test_split
from scipy.stats import linregress
from itertools import product
import pyprind

import sys

n_pcs = range(1, 82, 20)
non_lin = [1, 2]

alpha = 0
nlag = 2

#empty dictionary of the best model we've fit for each cell
best_model = {k: (-np.inf, None) for k in data.keys()}

#make a dictionary of dictionaries to store the results
scores_dict = {k: {} for k in data.keys()}

bar = pyprind.ProgBar(len(n_pcs)*len(non_lin))

for (l, n) in product(non_lin, n_pcs):
    bar.update()
    
    lagged_stim = lagmat(stim[:, 0:n], maxlag = nlag, trim = None)[1:-1]       
    stim_array = np.concatenate((lagged_stim, run_speed[:, np.newaxis]), axis = 1)
    
    
    for c in data.keys():
        X_train, X_test, y_train, y_test = train_test_split(stim_array, np.array(data[c]))     
        weight_init = np.linalg.pinv(X_train).dot(y_train)[:, np.newaxis]
       
        if l == 1:
            nonlin = tf.nn.sigmoid
        if l == 2:
            nonlin = tf.nn.relu

        
        model = gm.gaussian_GLM(weight_init, non_lin = nonlin, verbose = False, alpha = alpha)
        cv, loss = model.fit(X_train, y_train[:, np.newaxis], X_test, y_test[:, np.newaxis], max_iters = 150) 
        w, o, s  = model.get_params()

    
        _, _, r2_test,_, _ = linregress(np.squeeze(model.predict_trace(X_test)), np.squeeze(y_test))
        _, _, r2_train,_, _ = linregress(np.squeeze(model.predict_trace(X_train)), np.squeeze(y_train))

        scores_dict[c][(l, n)] = (r2_test, r2_train) 

        if r2_test > best_model[c][0]:
            best_model[c] = (r2_test, (l, n))    


0%      100%
[#         ] | ETA: 00:00:00

In [14]:
for c in data.keys():
    
    if best_model[c][1] != None:
        (v, (l, a, n)) = best_model[c]
    
    print 'explained var: ', round(v, 3), 'non_lin:', l, 'n_components', n

explained var:  0.039 non_lin: 1 n_components 10
explained var:  0.003 non_lin: 1 n_components 10
explained var:  0.003 non_lin: 1 n_components 10
explained var:  0.003 non_lin: 1 n_components 10
explained var:  0.003 non_lin: 1 n_components 10
explained var:  0.003 non_lin: 1 n_components 10
explained var:  0.003 non_lin: 1 n_components 10
explained var:  0.003 non_lin: 1 n_components 10
explained var:  0.003 non_lin: 1 n_components 10
explained var:  0.003 non_lin: 1 n_components 10
explained var:  0.003 non_lin: 1 n_components 10
explained var:  0.003 non_lin: 1 n_components 10
explained var:  0.003 non_lin: 1 n_components 10
explained var:  0.003 non_lin: 1 n_components 10
explained var:  0.003 non_lin: 1 n_components 10
explained var:  0.003 non_lin: 1 n_components 10
explained var:  0.003 non_lin: 1 n_components 10
explained var:  0.003 non_lin: 1 n_components 10
explained var:  0.003 non_lin: 1 n_components 10
explained var:  0.003 non_lin: 1 n_components 10
explained var:  0.00